In [1]:
import torch
import torch.nn as nn
import numpy as np

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'


In [3]:
class Arguments:
    res_path = './results'
    model_path = './models'
    epoch  = 20
    patience = -1
    teacher = False
    bidi = False
    test = False
    shrd_dec_emb = False 
    btstrp  = None 
    lm = False 
    toy = False 
    pretty = False 
    mmi = False
    seq2seq = False 
    drp = 0.3
    num_lyr = 1
    lr = 0.0001
    bt_siz = 64
    beam = 1
    vocab_size = 50005 
    response_vocab_size = 32149
    emb_size = 300
    hid_size = 128
    ut_hid_size = 128
    ses_hid_size = 256
    dec_hid_size = 256
    kb_embed_size = 100
    kb_size = 9274341
    kb_relations_size= 571
    use_embed = True




In [4]:
options = Arguments()

In [5]:

ent_embed = np.load('transe_dir/ent_embed.pkl.npy')
rel_embed = np.load('transe_dir/rel_embed.pkl.npy')

new_row = np.zeros((1,100), dtype=np.float32)
    
ent_embed = np.vstack([new_row, ent_embed]) # corr. to <pad_kb>
ent_embed = np.vstack([new_row, ent_embed]) # corr. to <nkb>

rel_embed = np.vstack([new_row, rel_embed]) # corr. to <pad_kb>
rel_embed = np.vstack([new_row, rel_embed]) 



In [6]:
def mini_batch_processing(mini_batch):
    mem_size = len(mini_batch[0][6].split('|'))
    if mem_size > 10000 :
        mem_size =  10000
    u1_text = np.array([s[0][0] for s in mini_batch])
    u1_kb   = np.array([s[1][0] for s in mini_batch])
    u2_text = np.array([s[0][1] for s in mini_batch] )
    u2_kb   = np.array([s[1][1] for s in mini_batch])
    target_kb = np.array([s[2] for s in mini_batch])
    target_utterance = np.array([s[3] for s in mini_batch])
    cand_ent =  [ [int(i) for i in s[6].split('|')] for s in mini_batch]
    cand_ent =  [ s[:-1][:mem_size-1]+[1]  for s in cand_ent]
    cand_ent =  np.array([ s + [0]*(mem_size-len(s)) for s in cand_ent])
    
    cand_rel =  [ [int(i) for i in s[7].split('|')] for s in mini_batch]
    cand_rel =  [ s[:-1][:mem_size-1]+[1]  for s in cand_rel]
    cand_rel =  np.array([ s + [0]*(mem_size - len(s)) for s in cand_rel])
    
    cand_val =  [ [int(i) for i in s[8].split('|')] for s in mini_batch]
    cand_val =  [ s[:-1][:mem_size-1]+[1]  for s in cand_val]
    cand_val =  np.array([s + [0]*(mem_size-len(s)) for s in cand_val])
    '''
    mem_weights = ((cand_val!=0) * (cand_val!=1) ).astype(int)
    
   
    labels = []
    candidates = []
    for i in range(64):
        positive_samples_i =list( set(target_kb[i])-{0})
        labels_i = [1]*len(positive_samples_i)
        negative_samples_i = list(set(cand_val[i])-(set(target_kb[i])-{0}) )
        if len(negative_samples_i) > 0:
            negative_samples_subset_i  = list( np.random.choice(negative_samples_i,20 - len(labels_i)) )
        else :
            negative_samples_subset_i =[]

        labels_i = labels_i + [0] *(20 - len(labels_i))
        candidates_i = positive_samples_i + negative_samples_subset_i
        candidates_i = candidates_i + (20-len(candidates_i)) *[0]

        labels.append(labels_i)
        candidates.append( candidates_i )
    
    c = np.array(labels)
    candidates_pred = np.array(candidates)
    
    '''
    
    b , l = cand_val.shape
    c = np.zeros((b, l))
    for i in range(b):
        for j in range(l):
            c[i,j] = int( (cand_val[i,j] in target_kb[i]) and cand_val[i,j] != 0) 
    
    
    
    return(u1_text, u1_kb, u2_text, u2_kb, target_kb, target_utterance,  cand_ent, cand_rel, cand_val,c)


In [7]:
from model import  *

In [8]:
import os

In [9]:
def padded_candidates(t):
    n_samples = len(t)
    
    for i in range(n_samples):
        cand = t[i][6].split('|')
        n_cand = len(cand)
        if n_cand  < 10:
            t[i][6] = '|'.join( cand + ['0']*(10-n_cand))
            cand = t[i][7].split('|')
            t[i][7] = '|'.join( cand + ['0']*(10-n_cand))
            cand = t[i][8].split('|')
            t[i][8] = '|'.join( cand + ['0']*(10-n_cand))
    return(t)

In [10]:
import pickle as pkl

In [11]:
model_ = model(options)


/home/tony/miniconda2/lib/python2.7/site-packages/torch/nn/modules/rnn.py:38: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [12]:
model_.load_state_dict(torch.load('train.pt'))

In [13]:
model_.eval()

model(
  (enc): UtteranceEncoder(
    (drop): Dropout(p=0.3)
    (embed_vocab): Embedding(50005, 300)
    (rnn): GRU(400, 128, batch_first=True, dropout=0.3)
  )
  (kvmlookup): KVMemoryReader(
    (C): Linear(in_features=128, out_features=200, bias=True)
    (R): Linear(in_features=100, out_features=128, bias=True)
    (softmax): Softmax()
    (drop): Dropout(p=0.3)
  )
  (inter_utter_encoder): InterUtteranceEncoder(
    (rnn): GRU(128, 128, batch_first=True, dropout=0.3)
  )
  (dec): Decoder(
    (drop): Dropout(p=0.3)
    (tanh): Tanh()
    (embed_in): Embedding(32149, 300)
    (embed_out): Linear(in_features=300, out_features=32149, bias=False)
    (rnn): GRU(300, 256, batch_first=True, dropout=0.3)
    (ses_to_dec): Linear(in_features=256, out_features=256, bias=True)
    (dec_inf): Linear(in_features=256, out_features=600, bias=False)
    (ses_inf): Linear(in_features=256, out_features=600, bias=False)
    (emb_inf): Linear(in_features=300, out_features=600, bias=True)
  )
  (kg_p

In [14]:
from torch import optim
from tqdm import tqdm_notebook

In [15]:
testing_files=["dump/test_preprocessed/"+x for x in os.listdir("/home/tony/QA/dump/test_preprocessed/") if x.startswith('test')]

In [26]:
testing_files

['dump/test_preprocessed/test_data_file_comparative_count.pkl',
 'dump/test_preprocessed/test_data_file_logical.pkl',
 'dump/test_preprocessed/test_data_file_quantitative.pkl',
 'dump/test_preprocessed/test_data_file_comparative.pkl',
 'dump/test_preprocessed/test_data_file_simple.pkl',
 'dump/test_preprocessed/test_data_file_quantitative_count.pkl',
 'dump/test_preprocessed/test_data_file_verify.pkl']

In [27]:
test_kb = [testing_files[-3], testing_files[1], testing_files[2], testing_files[3]]
test_kb

['dump/test_preprocessed/test_data_file_simple.pkl',
 'dump/test_preprocessed/test_data_file_logical.pkl',
 'dump/test_preprocessed/test_data_file_quantitative.pkl',
 'dump/test_preprocessed/test_data_file_comparative.pkl']

In [28]:
test_word = [testing_files[0], testing_files[-2], testing_files[-1]]
test_word

['dump/test_preprocessed/test_data_file_comparative_count.pkl',
 'dump/test_preprocessed/test_data_file_quantitative_count.pkl',
 'dump/test_preprocessed/test_data_file_verify.pkl']

In [17]:
t = pkl.load(open(testing_files[-3], 'rb'))

In [18]:
id_ent_map = pkl.load(open('transe_dir/id_rel_map.pickle', 'rb'))
ent_id_map = dict( [(i[1], i[0]+2)for i in id_ent_map.items()] )
ent_id_map['nkb'] = 1
ent_id_map['pad'] = 0

In [19]:
model_.dec.teacher_forcing = False

In [20]:
model_.dec.tc_ratio = -100000

In [21]:
model_.cuda()

model(
  (enc): UtteranceEncoder(
    (drop): Dropout(p=0.3)
    (embed_vocab): Embedding(50005, 300)
    (rnn): GRU(400, 128, batch_first=True, dropout=0.3)
  )
  (kvmlookup): KVMemoryReader(
    (C): Linear(in_features=128, out_features=200, bias=True)
    (R): Linear(in_features=100, out_features=128, bias=True)
    (softmax): Softmax()
    (drop): Dropout(p=0.3)
  )
  (inter_utter_encoder): InterUtteranceEncoder(
    (rnn): GRU(128, 128, batch_first=True, dropout=0.3)
  )
  (dec): Decoder(
    (drop): Dropout(p=0.3)
    (tanh): Tanh()
    (embed_in): Embedding(32149, 300)
    (embed_out): Linear(in_features=300, out_features=32149, bias=False)
    (rnn): GRU(300, 256, batch_first=True, dropout=0.3)
    (ses_to_dec): Linear(in_features=256, out_features=256, bias=True)
    (dec_inf): Linear(in_features=256, out_features=600, bias=False)
    (ses_inf): Linear(in_features=256, out_features=600, bias=False)
    (emb_inf): Linear(in_features=300, out_features=600, bias=True)
  )
  (kg_p

In [34]:
count_rec = 0
count_prec  = 0
recall_ = 0
precision_ = 0
count_accuracy = 0
accuracy_ = 0 
for file in testing_files:
        t = pkl.load(open(file, 'rb'))
        t = padded_candidates(t)
        np.random.seed(0)
        np.random.shuffle(t)
        n_batches = (len(t)//64)
        t = sorted(t, key = lambda x : -1 *len(x[6].split('|')))
        
        for i in tqdm_notebook(range(0, n_batches+1)):
            
            minibatch = t[ (i) * 64 : (i+1) *64]
            
            u1_text, u1_kb, u2_text, u2_kb, target_kb, target_utterance,\
                                cand_ent, cand_rel, cand_val, c = mini_batch_processing(minibatch)
            
            
            u1_text = np.flip(u1_text, axis = 1).copy()
            u1_kb = np.flip(u1_kb, axis = 1).copy()

            u2_text = np.flip(u2_text, axis = 1).copy()
            u2_kb = np.flip(u2_kb, axis = 1).copy()

            u1_text = torch.LongTensor(u1_text).cuda()
            u2_text = torch.LongTensor(u2_text).cuda()

            u1_kb = torch.FloatTensor(ent_embed[u1_kb]).cuda()
            u2_kb = torch.FloatTensor(ent_embed[u2_kb]).cuda()
            system_utterance = target_utterance
            target_utterance = torch.LongTensor(target_utterance).cuda()
            
            cand_ent_embed  = torch.FloatTensor(ent_embed[cand_ent]).cuda()
            cand_rel_embed  = torch.FloatTensor(rel_embed[cand_rel]).cuda()
            cand_val_embed  = torch.FloatTensor(ent_embed[cand_val]).cuda()
            cand_pred_embed = torch.FloatTensor(ent_embed[cand_val]).cuda()

            pred = model_(u1_text, u1_kb, u2_text, u2_kb,target_utterance,\
                            cand_ent_embed, cand_rel_embed, cand_val_embed)
            
            pred_text = pred[0]
            _, pos = torch.topk(pred_text,1, dim=2)
            pos.squeeze_(dim=2) 
            pos = np.array(pos.tolist())
            
            if file in test_kb :
                for s_i in range(pos.shape[0]):
                    n_kb = sum(pos[s_i]==4)
                    cand_pred = set([])
                    if n_kb:
                        _, cand_pred = torch.topk(pred[1][s_i], n_kb)
                        cand_pred =set(cand_val[s_i][cand_pred.tolist()])

                    true_kbs = set( minibatch[s_i][-1].split('|') )
                    true_kbs = set([ent_id_map[i] for i in true_kbs if i in ent_id_map ])

                    if len(true_kbs) > 0 :
                        recall_  = recall_ + len(true_kbs & cand_pred)* 1./ len(true_kbs)
                        count_rec = count_rec + 1
                    if n_kb :
                        precision_ = precision_ + len(true_kbs & cand_pred)* 1./ ( len(cand_pred) )
                        count_prec = count_prec + 1
                    elif n_kb == 0 and len(true_kbs)> 0 :
                        count_prec = count_prec + 1

            else :
                
                accuracy_ = accuracy_ + (system_utterance[:, 1] == pos[:, 0]).sum()
                count_accuracy = count_accuracy + pos.shape[0]
                    
            
            
        
            


MemoryError: 

In [35]:
len(cand_ent)

64

In [36]:
recall_/count_rec

0.09953776662599748

In [37]:
precision_ / count_prec

0.11737840805424363

In [ ]:
accuracy_

In [39]:
count_accuracy

44243